In [11]:
import pandas as pd
import yaml
import argparse
import torch
from model.model_viz import GoalNet

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Some hyperparameters and settings

In [13]:
CONFIG_FILE_PATH = 'config/jaad.yaml'  # yaml config file containing all the hyperparameters
DATASET_NAME = 'jaad'

TEST_DATA_PATH = 'data/JAAD/test_jaad.pkl'
TEST_IMAGE_PATH = 'data/JAAD/test'
OBS_LEN = 15  # in timesteps
PRED_LEN = 45  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a

ROUNDS = 1  # How often to evaluate the whole dataset, get stochastic mean results
BATCH_SIZE = 1

#### Load config file and print hyperparameters

In [14]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'resize': 0.25,
 'batch_size': 8,
 'train_mode': 'scene',
 'matrics': 'mse',
 'learning_rate': 0.0001,
 'num_epochs': 50,
 'grad_accum': True,
 'batch_size_gpu': 1,
 'data_augmentation': ['rot', 'flip'],
 'encoder_channels': [32, 32, 64, 64, 64],
 'decoder_channels': [64, 64, 64, 32, 32],
 'waypoints': [44],
 'feature_extract': True,
 'segmentation_model': None,
 'bbox_wh_model': 'pretrained_models/bbox_wh/jaad_wh-125.pth',
 'feature_classes': 6,
 'temperature': 1.0,
 'loss_scale': 1000,
 'kernlen': 31,
 'nsig': 4,
 'clip_grad': False,
 'max_norm': 20,
 'use_TTST': True,
 'rel_threshold': 0.0001,
 'use_CWS': False,
 'CWS_params': {'sigma_factor': 6, 'ratio': 2, 'rot': True}}

#### Load preprocessed Data

In [15]:
df_test = pd.read_pickle(TEST_DATA_PATH)

In [16]:
df_test.head()

,frame,trackId,x,y,w,h,sceneId,metaId
0,00026,0,1112.0,741.5,46.0,113.0,video_0005-00026,0
1,00026,0,1112.0,741.5,46.0,113.0,video_0005-00026,0
2,00026,0,1112.5,741.5,47.0,113.0,video_0005-00026,0
3,00026,0,1112.5,741.5,47.0,113.0,video_0005-00026,0
4,00026,0,1112.5,741.5,47.0,113.0,video_0005-00026,0


#### Initiate model and load pretrained weights

In [17]:
model = GoalNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

In [18]:
model.load('pretrained_models/jaad_adam_final_12_weights.pt')

<All keys matched successfully>


#### Evaluate model

In [19]:
model.evaluate(df_test.loc[df_test['sceneId'] == 'video_0339-00112'][:60], params, image_path=TEST_IMAGE_PATH, batch_size=BATCH_SIZE, rounds=ROUNDS, 
               num_goals=NUM_GOALS, num_traj=NUM_TRAJ, device=None, dataset_name=DATASET_NAME)

Preprocess data


Prepare Dataset: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


Start testing


Round: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.29it/s]

In [20]:
model.evaluate(df_test.loc[df_test['sceneId'] == 'video_0344-00042'][60:120], params, image_path=TEST_IMAGE_PATH, batch_size=BATCH_SIZE, rounds=ROUNDS, 
               num_goals=NUM_GOALS, num_traj=NUM_TRAJ, device=None, dataset_name=DATASET_NAME)

Preprocess data


Prepare Dataset: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1004.14it/s]


Start testing


Round: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.47it/s]